In [2]:
import pandas as pd

In [10]:
df = pd.read_csv("data/reshaped/2017.csv").drop("Unnamed: 0", axis=1)
df

,Team,Name,Rating,Score,Home_Away,Opponent,Week,Score_1_weeks_ago,Score_2_weeks_ago,Score_3_weeks_ago,...,Score_8_weeks_ago,Score_9_weeks_ago,Score_10_weeks_ago,Score_11_weeks_ago,Score_12_weeks_ago,Score_13_weeks_ago,Score_14_weeks_ago,Score_15_weeks_ago,Score_16_weeks_ago,Score_17_weeks_ago
0,Packers,Aaron Rodgers,98,16.5,Home,81,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Packers,Aaron Rodgers,98,18.5,Away,87,2,16.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Packers,Aaron Rodgers,98,24.8,Home,74,3,18.5,16.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Packers,Aaron Rodgers,98,23.1,Home,72,4,24.8,18.5,16.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Packers,Aaron Rodgers,98,24,Away,91,5,23.1,24.8,18.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168,Bills,Tyrod Taylor,82,3.8,Home,94,13,14,18.1,2.9,...,8.9,12.5,17.8,10.5,18.8,NaN,NaN,NaN,NaN,NaN
1169,Bills,Tyrod Taylor,82,-,Home,72,14,3.8,14,18.1,...,-,8.9,12.5,17.8,10.5,18.8,NaN,NaN,NaN,NaN
1170,Bills,Tyrod Taylor,82,23.2,Home,70,15,-,3.8,14,...,20,-,8.9,12.5,17.8,10.5,18.8,NaN,NaN,NaN
1171,Bills,Tyrod Taylor,82,12.8,Away,94,16,23.2,-,3.8,...,16.7,20,-,8.9,12.5,17.8,10.5,18.8,NaN,NaN
